In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import h5py
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Dataset importation

In [2]:
dir_output = "Output"
features_path = dir_output + "/features.h5"
labels_path = dir_output + "/labels.h5"
test_size = 0.3

# import features and labels
h5f_data = h5py.File(features_path, 'r')
h5f_label = h5py.File(labels_path, 'r')

features_string = h5f_data['dataset_skin_lesion']
labels_string = h5f_label['dataset_skin_lesion']

X = np.array(features_string)
Y = np.array(labels_string)

h5f_data.close()
h5f_label.close()

# SPLIT DATA INTO TRAINING AND TEST SETS
(X_train, X_test, Y_train, Y_test) = train_test_split(X, Y,
                                                      test_size=test_size,
                                                      random_state=458773245,
                                                      stratify=Y)

print(np.unique(Y, return_counts=True))
print(np.unique(Y_train, return_counts=True))
print(np.unique(Y_test, return_counts=True))

# # STANDARDIZE DATA
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.from_numpy(X_train).float().unsqueeze(1)
X_test_tensor = torch.from_numpy(X_test).float().unsqueeze(1)
Y_train_tensor = torch.from_numpy(Y_train).float() 
Y_test_tensor = torch.from_numpy(Y_test).float()

train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=5, shuffle=False)

(array([0., 1.]), array([100, 100]))
(array([0., 1.]), array([70, 70]))
(array([0., 1.]), array([30, 30]))


# Convolutional Model

In [3]:
class Conv_sig(nn.Module):
    
    def __init__(self):
        super(Conv_sig, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 3, 1, 2)
        self.conv2 = nn.Conv1d(16, 32, 3, 1, 2)
        self.dropout1 = nn.Dropout1d(0.2)
        self.dropout2 = nn.Dropout1d(0.2)
        self.linear1 = nn.Linear(2880, 256)
        self.linear2 = nn.Linear(256, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = F.sigmoid(x)
        x = F.max_pool1d(x, 3)
        x = self.dropout1(x)

        x = self.conv2(x)
        x = F.sigmoid(x)
        x = F.max_pool1d(x, 3)
        x = self.dropout1(x)

        x = torch.flatten(x, 1)
        x = self.linear1(x)
        x = F.sigmoid(x)
        x = self.dropout2(x)
        x = self.linear2(x)

        output = F.sigmoid(x)
        return output
    
class Conv_relu(nn.Module):
    
    def __init__(self):
        super(Conv_relu, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 3, 1, 2)
        self.conv2 = nn.Conv1d(16, 32, 3, 1, 2)
        self.dropout1 = nn.Dropout1d(0.2)
        self.dropout2 = nn.Dropout1d(0.2)
        self.linear1 = nn.Linear(2880, 256)
        self.linear2 = nn.Linear(256, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool1d(x, 3)
        x = self.dropout1(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool1d(x, 3)
        x = self.dropout1(x)

        x = torch.flatten(x, 1)
        x = self.linear1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.linear2(x)

        output = F.sigmoid(x)
        return output
    
class Conv_tanh(nn.Module):
    
    def __init__(self):
        super(Conv_tanh, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 3, 1, 2)
        self.conv2 = nn.Conv1d(16, 32, 3, 1, 2)
        self.dropout1 = nn.Dropout1d(0.2)
        self.dropout2 = nn.Dropout1d(0.2)
        self.linear1 = nn.Linear(2880, 256)
        self.linear2 = nn.Linear(256, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = F.tanh(x)
        x = F.max_pool1d(x, 3)
        x = self.dropout1(x)

        x = self.conv2(x)
        x = F.tanh(x)
        x = F.max_pool1d(x, 3)
        x = self.dropout1(x)

        x = torch.flatten(x, 1)
        x = self.linear1(x)
        x = F.tanh(x)
        x = self.dropout2(x)
        x = self.linear2(x)

        output = F.sigmoid(x)
        return output

# Linear model

In [4]:
class Lin_sig(nn.Module):
    def __init__(self):
        super(Lin_sig, self).__init__()
        self.linear1 = nn.Linear(803, 512)
        self.linear2 = nn.Linear(512, 128)
        self.linear3 = nn.Linear(128, 32)
        self.linear4 = nn.Linear(32, 1)
        self.dropout = nn.Dropout1d(0.2)


    def forward(self, x):
        x = self.linear1(x)
        x = F.sigmoid(x)
        x = self.dropout(x)

        x = self.linear2(x)
        x = F.sigmoid(x)
        x = self.dropout(x)

        x = self.linear3(x)
        x = F.sigmoid(x)
        x = self.dropout(x)

        x = self.linear4(x)
        output = F.sigmoid(x)
        return output
    
class Lin_relu(nn.Module):
    
    def __init__(self):
        super(Lin_relu, self).__init__()
        self.linear1 = nn.Linear(803, 512)
        self.linear2 = nn.Linear(512, 128)
        self.linear3 = nn.Linear(128, 32)
        self.linear4 = nn.Linear(32, 1)
        self.dropout = nn.Dropout1d(0.2)


    def forward(self, x):
        x = self.linear1(x)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.linear2(x)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.linear3(x)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.linear4(x)
        output = F.sigmoid(x)
        return output
    
class Lin_tanh(nn.Module):
    
    def __init__(self):
        super(Lin_tanh, self).__init__()
        self.linear1 = nn.Linear(803, 512)
        self.linear2 = nn.Linear(512, 128)
        self.linear3 = nn.Linear(128, 32)
        self.linear4 = nn.Linear(32, 1)
        self.dropout = nn.Dropout1d(0.2)


    def forward(self, x):
        x = self.linear1(x)
        x = F.tanh(x)
        x = self.dropout(x)

        x = self.linear2(x)
        x = F.tanh(x)
        x = self.dropout(x)

        x = self.linear3(x)
        x = F.tanh(x)
        x = self.dropout(x)

        x = self.linear4(x)
        output = F.sigmoid(x)
        return output

# Training

In [72]:
# Initialisation 
model = Conv_tanh()
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=5e-4)

# Training
num_epochs = 500

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs).squeeze()  
        loss = criterion(outputs, labels)  
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

Epoch 1/500, Loss: 0.8381
Epoch 2/500, Loss: 0.6244
Epoch 3/500, Loss: 0.6456
Epoch 4/500, Loss: 0.6253
Epoch 5/500, Loss: 0.6544
Epoch 6/500, Loss: 0.6530
Epoch 7/500, Loss: 0.6350
Epoch 8/500, Loss: 0.6289
Epoch 9/500, Loss: 0.5945
Epoch 10/500, Loss: 0.5853
Epoch 11/500, Loss: 0.5776
Epoch 12/500, Loss: 0.6165
Epoch 13/500, Loss: 0.5909
Epoch 14/500, Loss: 0.5930
Epoch 15/500, Loss: 0.5718
Epoch 16/500, Loss: 0.6131
Epoch 17/500, Loss: 0.5674
Epoch 18/500, Loss: 0.5648
Epoch 19/500, Loss: 0.5574
Epoch 20/500, Loss: 0.5761
Epoch 21/500, Loss: 0.5717
Epoch 22/500, Loss: 0.5548
Epoch 23/500, Loss: 0.5448
Epoch 24/500, Loss: 0.5258
Epoch 25/500, Loss: 0.5251
Epoch 26/500, Loss: 0.5579
Epoch 27/500, Loss: 0.5285
Epoch 28/500, Loss: 0.5318
Epoch 29/500, Loss: 0.5428
Epoch 30/500, Loss: 0.5485
Epoch 31/500, Loss: 0.5320
Epoch 32/500, Loss: 0.5295
Epoch 33/500, Loss: 0.5114
Epoch 34/500, Loss: 0.5292
Epoch 35/500, Loss: 0.5392
Epoch 36/500, Loss: 0.5631
Epoch 37/500, Loss: 0.5392
Epoch 38/5

# Evaluation

In [77]:
def evaluate(model, test_loader):
    model.eval()
    y_pred = []
    y_true = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs).squeeze()
            print(outputs)
            #print(labels)
            y_pred.extend((outputs > 5e-1).float().cpu().numpy())
            y_true.extend(labels.cpu().numpy())

    return y_pred, y_true

In [78]:
y_pred, y_true = evaluate(model, test_loader)
print(confusion_matrix(y_true, y_pred), classification_report(y_true, y_pred))
accuracy_score(y_true, y_pred)

tensor([0.9731, 0.9968, 0.9531, 0.9999, 0.0032])
tensor([2.2653e-07, 1.5950e-04, 1.6195e-01, 4.9029e-04, 9.9957e-01])
tensor([2.0159e-02, 2.6646e-02, 9.6645e-01, 6.1273e-07, 9.9680e-01])
tensor([2.1621e-03, 1.3147e-05, 9.7780e-01, 9.9988e-01, 9.0926e-01])
tensor([5.3425e-06, 9.9459e-01, 9.9868e-01, 6.8783e-01, 2.5895e-02])
tensor([0.9999, 0.9997, 0.1362, 0.0033, 0.9985])
tensor([9.9974e-01, 2.0724e-01, 1.1791e-05, 8.7373e-01, 2.5999e-03])
tensor([1.0158e-06, 1.8096e-05, 8.6401e-06, 9.9982e-01, 9.9998e-01])
tensor([0.9971, 0.0661, 0.7232, 0.1222, 0.0014])
tensor([9.9938e-01, 2.0902e-04, 3.3481e-06, 3.8144e-03, 9.8403e-01])
tensor([9.9384e-01, 5.8010e-01, 1.0000e+00, 9.9545e-01, 5.5365e-05])
tensor([0.0019, 0.2014, 0.0274, 0.9651, 0.3353])
[[23  7]
 [ 8 22]]               precision    recall  f1-score   support

         0.0       0.74      0.77      0.75        30
         1.0       0.76      0.73      0.75        30

    accuracy                           0.75        60
   macro avg   

0.75